In [17]:
import cv2
import os
import face_recognition
import pandas as pd
import numpy as np
import cvzone
import datetime
import mysql.connector

# importing all modules

In [18]:
# Entering the student's info
def insert_students_data(studentid,name,branch,roll_no,year,picpath,section):
    import mysql
    import mysql.connector
    import os
    import cv2
    import face_recognition
    import pickle

# Connect to the database
    mydb = mysql.connector.connect(
      host="localhost",
      user="your sql user ",
      password="your sql pwd",
      database="facial_recognition_based_attendance_system1"
    )
    with open(picpath,"rb") as f:
        picture_data=f.read()
    # Create a cursor object
    cor=cv2.imread(picpath)
    img=cv2.cvtColor(cor,cv2.COLOR_BGR2RGB)
    
    temp=face_recognition.face_encodings(img)[0]
    
    
    file=open(r"Give a suitable destination\EncodeFilet.p","wb")
    pickle.dump([temp],file)
    file.close()
    
    with open(r"Give a suitable destination\EncodeFilet.p","rb") as f1:
        encode_data=f1.read()
    f1.close()
   
   
    mycursor = mydb.cursor()

    # Define the SQL query
    sql = "INSERT INTO students (student_id, name, branch,roll_no,year,pic,section,encodedfile) VALUES (%s, %s, %s, %s, %s, %s, %s,%s)"
    values = (studentid,name,branch,roll_no,year,picture_data,section,encode_data)

    # Execute the query
    mycursor.execute(sql, values)

    # Commit the changes
    mydb.commit()
    print("Database updated")
    os.remove(picpath)
    os.remove(r"Give a suitable destination\EncodeFilet.p")


In [19]:
# Creating a class

def create_class(class_no):
    import mysql.connector
    mydb = mysql.connector.connect(
      host="localhost",
      user="your user",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="create table "+str(class_no)+"_"+"(student_id varchar(30),attendance_time datetime)"
    mycursor=mydb.cursor()
    mycursor.execute(sql)
    mydb.commit()
    mydb.commit()
    print("class successfully initiated")



In [20]:
# marking attendance

def mark_attendance(student_id,attendance_time,classno):
    import mysql.connector
    from datetime import datetime
    mydb = mysql.connector.connect(
      host="localhost",
      user="your user",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="INSERT INTO "+str(classno)+"_"+" (student_id,attendance_time) VALUES (%s, %s)"
    values=(student_id,attendance_time)
    mycursor=mydb.cursor()
    mycursor.execute(sql,values)
    mydb.commit() 



In [21]:

def creating_students_table():
    import mysql.connector
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="create table students(student_id varchar(30) primary key,name varchar(30),roll_no int,year int default 1,branch varchar(20),pic longblob,section char(1),encodedfile longblob)"
    mycursor=mydb.cursor()
    mycursor.execute(sql)
    mydb.commit()
    
    print("table successfully initiated")


In [22]:

def show_attendance(classno):
    import mysql.connector
    from tabulate import tabulate
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="select students.student_id,students.name,students.section,students.branch,students.roll_no,"+classno+"_.attendance_time from students,"+classno+"_ where students.student_id="+classno+"_.student_id"
   
    mycursor=mydb.cursor(buffered=True)
    mycursor.execute(sql)
    mydb.commit() 
    
    rows = mycursor.fetchall()

    # Get the column names
    col_names = [i[0] for i in mycursor.description]

    # Print the table using tabulate
    print(tabulate(rows, headers=col_names, tablefmt='psql'))
    
    

    

In [23]:



def show_attendance1(classno,student_id):
    import mysql.connector
    from tabulate import tabulate
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="select attendance_time from "+classno+"_ where student_id="+"'"+str(student_id)+"'"
   
    mycursor=mydb.cursor(buffered=True)
    mycursor.execute(sql)
    result=mycursor.fetchone()
    if result:
        return False
    else:
        return True


    

In [24]:

def getdata():
    import mysql.connector
    import pickle
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="your password",
      database="facial_recognition_based_attendance_system1"
    )
    sql="select * from students"
   
    mycursor=mydb.cursor(buffered=True)
    mycursor.execute(sql)
    mydb.commit() 
    
    rows = mycursor.fetchall()
    
    studentall=[]
    studentInfo={}
    encodeListKnown=[]
    studentId=[]
    for i in rows:
        studentId+=[i[0]]
        encodeListKnown+=[pickle.loads(i[-1])[0]]
    for i in rows:
        studentInfo["student_id"]=i[0]
        studentInfo["name"]=i[1]
        studentInfo["roll_no"]=i[2]
        studentInfo["year"]=i[3]
        studentInfo["branch"]=i[4]
        studentInfo["pic"]=i[5]
        studentInfo["section"]=i[6]
        studentInfo["encodedfile"]=i[7]
        studentall+=[studentInfo]
        studentInfo={}
    
    return studentId,studentall,encodeListKnown






In [25]:
import os
import cv2
imgModeList=[]
modePath=r"Give a destination file path\Modes"
modeList=os.listdir(modePath)
for i in modeList:
    temp=cv2.imread(modePath+"\\"+i)
    imgModeList.append(temp)

In [26]:
def find_correctid(sid):
    studentId,studentall,encodeListKnown=getdata()
    
    for i in studentall:
        if i["student_id"]==sid:
            studentInfo=i
            
            return studentInfo


In [27]:
def main():
    import os
    from datetime import datetime
    import face_recognition
    import pickle
    from PIL import Image
    studentId,studentall,encodeListKnown=getdata()

    cap=cv2.VideoCapture(0)
    # cap.set(3,width)
    # cap.set(4,height)
    cap.set(3,640)
    cap.set(4,480)
    background=cv2.imread(r"GIve a destination path \background.png")
    modeType=0

    attendence=dict.fromkeys(studentId,0)

    counter=0
    sid=-1
    classno=input("Please enter the class no to which you want to mark attendance for: ")


    imgStudent=[]
    studentInfo=[]

    while True:
        success,img=cap.read()

        imgS=cv2.resize(img,(0,0),None,0.25,0.25)
        imgS=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

        background[162:162 + 480, 55:55 + 640] = img
        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]


        faceCurFrame=face_recognition.face_locations(imgS)
        encodeCurFrame=face_recognition.face_encodings(imgS,faceCurFrame)


        if faceCurFrame:
            for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame):

                matches=face_recognition.compare_faces(encodeListKnown,encodeFace,tolerance=0.5)

                faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)

                matchIndex=np.argmin(faceDis)

                if matches[matchIndex]:
                    y1,x2,y2,x1=faceLoc
                    bbox=x1+55,y1+162,x2-x1,y2-y1
                    print("Known face detected")
                    sid=studentId[matchIndex]


                    if counter==0:
                        counter=1
                        modeType=1

                        cvzone.putTextRect(background,"Loading",(275,400))
                        cv2.imshow("img",background)
                        cv2.waitKey(1)
                result=show_attendance1(classno,sid)
                studentInfo=find_correctid(sid)
                if sid!=-1:
                    with open("student.jpg","wb") as f:
                        f.write(studentInfo["pic"])
                        cor=cv2.imread("student.jpg")
                    imgStudent=cv2.cvtColor(cor,cv2.COLOR_BGRA2BGR)

                if counter!=0:
                    if counter==1:


                        if result:
                            # standard meathod of conversion
                            mark_attendance(sid,datetime.now(),classno)




                        else:
                            modeType=3
                            background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                            imgStudent=[]
                            studentInfo=[]
                            counter=0
                    if modeType!=3:    
                        if 10<counter<20:
                            modeType=2
                            background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

                        if counter<=10:
                            modeType=1
                            background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

                            cv2.putText(background,str(studentInfo['branch']),(1003,550),cv2.FONT_HERSHEY_PLAIN,
                                       0.6,(255,255,255),1)
                            cv2.putText(background,str(sid),(1003,493),cv2.FONT_HERSHEY_PLAIN,
                                       0.6,(255,255,255),1)

                            cv2.putText(background,str(str(studentInfo['year'])+" Year"),(1007,625),cv2.FONT_HERSHEY_PLAIN,
                                       0.9,(100,100,100),1)


                            (w,h),_=cv2.getTextSize(studentInfo['name'],cv2.FONT_HERSHEY_PLAIN,1,1)
                            offset=int((414-w)/2)
                            cv2.putText(background,str(studentInfo['name']),(808+offset,445),cv2.FONT_HERSHEY_PLAIN,
                                       1,(50,50,50),1)

                            background[175:175+216,909:909+216]=imgStudent
                    counter+=1
                    if counter>=20:
                        modeType=0
                        counter=0
                        imgStudent=[]
                        studentInfo=[]
                        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
        else:

            modeType=0
            modeType=0
            counter=0
            imgStudent=[]
            studentInfo=[]
            background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]


        cv2.imshow("img",background)

        if cv2.waitKey(1) & 0xFF==ord('a'):
            break
    cap.release()
    cv2.destroyAllWindows()
    os.remove("student.jpg")

In [28]:
def student():
    print("\t\tYou have entered the student portal\n\n")
    main()
    

In [29]:
def teacher():
    x=int(input('''Please enter the following code for specific tasks
    1: (FOR FIRST TIME) Create students table
    2: Enter students data
    3: Initiate a new class
    4: check the attendane of a class'''))
    if x==1:
        creating_students_table()
    elif x==2:
        print("\t You have entered the student credential entry portal")
        print("*** Please note that the images must be of the size 216x216 pixels ***")
        student_id=input("Please enter the student id")
        name=input("Please enter the Name of the student")
        branch=input("Please enter the branch of the student")
        roll_no=int(input("Please enter the roll no of student"))
        year=int(input("Please enter the year of the student"))
        path=input("Please enter the path of the image of student without inverted commas")
        section=input("Please enter the section of the student")
        insert_students_data(student_id,name,branch,roll_no,year,path,section)
        print("Insertion complete")
    elif x==3:
        class_no=int(input("Please enter the class number (integer value)"))
        create_class(class_no)
    elif x==4:
        class_no=input("Please enter the class number (integer value)")
        show_attendance(class_no)
    else:
        print("Please enter a valid option")

In [ ]:
print("\t FACIAL RECOGNITION BASED ATTENDANCE SYSTEM\n\n\n")
y=int(input('''Please enter the following code for specific tasks
1: For student
2: For teachers
'''))
if y==1:
    print("\t You have opted for student option")
    student()
elif y==2:
    print("\t You have opted for teacher option")
    teacher()
else:
    print("Please enter a valid code")

	 FACIAL RECOGNITION BASED ATTENDANCE SYSTEM



Please enter the following code for specific tasks
1: For student
2: For teachers
1
	 You have opted for student option
		You have entered the student portal


Please enter the class no to which you want to mark attendance for: 1002
Known face detected
Known face detected
